In [1]:
import os
import tsl
import torch
import numpy as np
import pandas as pd
import folium
from torch_geometric.utils import dense_to_sparse, to_dense_adj

print(f"tsl version  : {tsl.__version__}")
print(f"torch version: {torch.__version__}")

pd.options.display.float_format = '{:.2f}'.format
np.set_printoptions(edgeitems=3, precision=3)
torch.set_printoptions(edgeitems=2, precision=3)

tsl version  : 0.9.5
torch version: 2.1.2


/home/filippo/miniconda3/envs/peakweather-env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Utility functions

In [2]:

# Utility functions ################
def print_matrix(matrix):
    return pd.DataFrame(matrix)

def print_model_size(model):
    tot = sum([p.numel() for p in model.parameters() if p.requires_grad])
    out = f"Number of model ({model.__class__.__name__}) parameters:{tot:10d}"
    print("=" * len(out))
    print(out)

def plot_map(dataset, ei, ew, filename, th=0.0, color_factor=1.0):
    """
    Plots the graph on a geographical map using Folium.

    Parameters:
    - dataset: The dataset containing sensor locations.
    - ei: Edge index (pairs of connected nodes).
    - ew: Edge weights (used to set line thickness/color).
    - filename: Name of the HTML file where the map will be saved.
    - th: Threshold for filtering weak connections.
    - color_factor: Factor to scale the visual weight.
    """

    #ei = ei.cpu().numpy()
    #ew = ew.cpu().numpy()
    pq_path = 'data/PeakWeather/stations.parquet'
    locations = pd.read_parquet(pq_path)
    if 'index' in locations.columns:
        locations = locations.set_index('index')
    nodes_df = locations.loc[:, ['latitude', 'longitude']]

    # Filter edges based on weight threshold
    ei = nodes_df.index.to_numpy()[ei[:, ew > th]]
    ew = ew[ew > th] * color_factor

    # Create a map centered around the first coordinate
    city_map = folium.Map(location=[nodes_df['latitude'].mean(), nodes_df['longitude'].mean()],
                            zoom_start=12, tiles='Esri.WorldStreetMap', attr="Simplified Map")


    for i in range(ei.shape[1]):
        # Get the coordinates of the start and end nodes
        start_node = nodes_df.loc[nodes_df.index == ei[0, i]]
        end_node = nodes_df.loc[nodes_df.index == ei[1, i]]
        if len(start_node) == 0 or len(end_node) == 0:
            pass
        else:
            coords = [
                [start_node['latitude'].values[0], start_node['longitude'].values[0]],
                [end_node['latitude'].values[0], end_node['longitude'].values[0]]
            ]
            folium.PolyLine(coords, color="blue", weight=float(ew[i]), opacity=float(ew[i])).add_to(city_map)

    # Add points to the map
    for _, row in nodes_df.iterrows():
        # folium.Marker(location=[row['latitude'], row['longitude']]).add_to(city_map)
        folium.CircleMarker(
            location=[row['latitude'], row['longitude']],
            radius=3,
            color='orange',
            fill=True,
            fill_opacity=1,
            popup=f"Node {row.index}"
        ).add_to(city_map)

    # Save the map to an HTML file
    city_map.save(filename)

# Dataset - Only meteo stations (no rain gauges)

In [3]:
from lib.datasets import PeakWeather
from tsl.data import SpatioTemporalDataset
from tsl.data.datamodule import (SpatioTemporalDataModule,
                                 TemporalSplitter)
from tsl.data.preprocessing import StandardScaler


Here we compute the original adjacency matrix based on sensor distances.
The weight between nodes i and j is given by:

\begin{split}
w^{i,j} = \left\{\begin{array}{cl}
     \exp \left(-\frac{\operatorname{dist}\left(i, j\right)^{2}}{\gamma}\right) & \operatorname{dist}\left(i, j\right) \leq \delta  \\
     0 & \text{otherwise}
\end{array}\right. ,
\end{split}


The function `dataset.get_connectivity()` computes this weighted connectivity.

In [4]:
dataset = PeakWeather(
            root="data/v1",
            target_channels=["temperature"],
            covariate_channels="other",
            station_type="meteo_station",
            freq="h",
            extended_topo_vars=["DEM", "SLOPE_2000M_SIGRATIO1"]) # elevation and slope
connectivity = dataset.get_connectivity(layout="edge_index",
                                     include_self=False,
                                     theta=50,
                                     threshold=0.1)

In [5]:
print(f"Has missing values: {dataset.has_mask}")
print(f"Percentage of missing values: {(1 - dataset.mask.mean()) * 100:.2f}%")
print(f"Has Covariates: {dataset.has_covariates}")
print(f"Covariates: {', '.join(dataset.covariates.keys())}")

Has missing values: True
Percentage of missing values: 7.09%
Has Covariates: True
Covariates: stations_table, installation_table, parameters_table, u, u_mask


In [6]:
dataset.dataframe()[:5]

nodes,ABO,AEG,AIG,ALT,AND,ANT,ARH,ARO,ATT,BAN,...,VAD,VEV,VIO,VIS,VIT,VLS,WAE,WFJ,WYN,ZER
channels,temperature,temperature,temperature,temperature,temperature,temperature,temperature,temperature,temperature,temperature,...,temperature,temperature,temperature,temperature,temperature,temperature,temperature,temperature,temperature,temperature
datetime,,,,,,,,,,,,,,,,,,,,,
2017-01-01 00:00:00+00:00,1.10,0.00,-4.70,-3.50,-6.20,-10.90,-3.70,-0.30,-3.50,0.00,...,-5.80,1.20,4.00,-8.40,-7.20,-5.00,-3.50,-3.20,-3.00,-3.50
2017-01-01 01:00:00+00:00,1.60,0.00,-5.50,-3.60,-5.30,-10.40,-3.50,0.50,-3.00,0.00,...,-6.70,1.20,3.80,-6.40,-7.30,-5.20,-3.40,-3.40,-3.20,-3.80
2017-01-01 02:00:00+00:00,0.40,0.00,-5.90,-3.70,-3.00,-10.30,-3.60,-0.40,-3.70,0.00,...,-6.90,1.10,3.40,-7.90,-7.30,-5.40,-3.00,-3.60,-3.10,-4.20
2017-01-01 03:00:00+00:00,1.10,0.00,-5.10,-3.80,-4.00,-10.60,-3.10,-0.60,-2.90,0.00,...,-7.30,1.10,3.00,-8.60,-7.30,-5.30,-3.30,-3.50,-3.20,-3.30
2017-01-01 04:00:00+00:00,0.40,0.00,-6.00,-4.20,-4.10,-10.40,-3.40,-0.50,-3.40,0.00,...,-7.40,0.50,3.00,-7.60,-7.30,-5.60,-3.60,-3.80,-3.10,-4.00


## plot interactive HTML map

In [7]:
edge_index, edge_weight = connectivity
plot_map(dataset, edge_index, edge_weight, 'peakweather_map.html', th=0)

### Transform to a pytorch compatible dataset (wrap with tsl)

In [ ]:
torch_dataset = SpatioTemporalDataset(target=dataset.dataframe(),
                                      connectivity=connectivity,
                                      mask=dataset.mask,
                                      horizon=24,
                                      window=6,
                                      stride=1)
print(torch_dataset)

SpatioTemporalDataset(n_samples=36130, n_nodes=160, n_channels=1)


### Preparing the dataset for training

In [9]:
# Normalize data using mean and std computed over time and node dimensions
scalers = {'target': StandardScaler(axis=(0, 1))}
# Split data sequentially:
#   |------------ dataset -----------|
#   |--- train ---|- val -|-- test --|
#TODO split data based on date (test starts on 1st april 2024)
splitter = TemporalSplitter(val_len=0.1, test_len=0.2)
datamodule = SpatioTemporalDataModule(
    dataset=torch_dataset,
    scalers=scalers,
    splitter=splitter,
    batch_size=64,
)

In [10]:
datamodule.setup()
print(datamodule)

{Train dataloader: size=52022}
{Validation dataloader: size=5774}
{Test dataloader: size=14452}
{Predict dataloader: None}


## Training setup

In [11]:
import torch.nn as nn

from tsl.nn.blocks.encoders import RNN
from tsl.nn.layers import NodeEmbedding, DiffConv
from einops.layers.torch import Rearrange


class TimeThenSpaceModel(nn.Module):
    def __init__(self, input_size: int, n_nodes: int, horizon: int,
                 hidden_size: int = 32,
                 rnn_layers: int = 1,
                 gnn_kernel: int = 2):
        super(TimeThenSpaceModel, self).__init__()

        self.encoder = nn.Linear(input_size, hidden_size)

        self.node_embeddings = NodeEmbedding(n_nodes, hidden_size)

        self.time_nn = RNN(input_size=hidden_size,
                           hidden_size=hidden_size,
                           n_layers=rnn_layers,
                           cell='gru',
                           return_only_last_state=True)

        self.space_nn = DiffConv(in_channels=hidden_size,
                                 out_channels=hidden_size,
                                 k=gnn_kernel)

        self.decoder = nn.Linear(hidden_size, input_size * horizon)
        self.rearrange = Rearrange('b n (t f) -> b t n f', t=horizon)

    def forward(self, x, edge_index, edge_weight):
        # x: [batch time nodes features]
        x_enc = self.encoder(x)  # linear encoder: x_enc = xΘ + b
        x_emb = x_enc + self.node_embeddings()  # add node-identifier embeddings
        h = self.time_nn(x_emb)  # temporal processing: x=[b t n h] -> h=[b n h]
        z = self.space_nn(h, edge_index, edge_weight)  # spatial processing
        x_out = self.decoder(z)  # linear decoder: z=[b n h] -> x_out=[b n t⋅f]
        x_horizon = self.rearrange(x_out) # x_out=[b n t⋅f] -> x_out=[b t n f]
        return x_horizon

In [12]:
hidden_size = 32   #@param
rnn_layers = 1     #@param
gnn_kernel = 2     #@param

input_size = torch_dataset.n_channels   # 1 channel
n_nodes = torch_dataset.n_nodes         # 207 nodes
horizon = torch_dataset.horizon         # 12 time steps

stgnn = TimeThenSpaceModel(input_size=input_size,
                           n_nodes=n_nodes,
                           horizon=horizon,
                           hidden_size=hidden_size,
                           rnn_layers=rnn_layers,
                           gnn_kernel=gnn_kernel)
print(stgnn)
print_model_size(stgnn)

TimeThenSpaceModel(
  (encoder): Linear(in_features=1, out_features=32, bias=True)
  (node_embeddings): NodeEmbedding(n_nodes=160, embedding_size=32)
  (time_nn): RNN(
    (rnn): GRU(32, 32)
  )
  (space_nn): DiffConv(32, 32)
  (decoder): Linear(in_features=32, out_features=24, bias=True)
  (rearrange): Rearrange('b n (t f) -> b t n f', t=24)
)
Number of model (TimeThenSpaceModel) parameters:     17464


## Predictor

In [13]:
from tsl.metrics.torch import MaskedMAE, MaskedMAPE
from tsl.engines import Predictor

loss_fn = MaskedMAE()

metrics = {'mae': MaskedMAE(),
           'mape': MaskedMAPE(),
           'mae_at_15': MaskedMAE(at=2),  # '2' indicates the third time step,
                                          # which correspond to 3 hours ahead
           'mae_at_30': MaskedMAE(at=5),
           'mae_at_60': MaskedMAE(at=11)}

# setup predictor
predictor = Predictor(
    model=stgnn,                   # our initialized model
    optim_class=torch.optim.Adam,  # specify optimizer to be used...
    optim_kwargs={'lr': 0.001},    # ...and parameters for its initialization
    loss_fn=loss_fn,               # which loss function to be used
    metrics=metrics                # metrics to be logged during train/val/test
)

## Train

In [14]:
from pytorch_lightning.loggers import MLFlowLogger
# mlflow ui --port <PORT>
logger = MLFlowLogger(experiment_name="temperature_prediction_test")

In [15]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks import ModelCheckpoint

checkpoint_callback = ModelCheckpoint(
    dirpath='logs',
    save_top_k=1,
    monitor='val_mae',
    mode='min',
)

trainer = pl.Trainer(max_epochs=50,
                     logger=logger,
                     accelerator="gpu" if torch.cuda.is_available() else "cpu",
                     devices=1,
                     callbacks=[checkpoint_callback])

trainer.fit(predictor, datamodule=datamodule)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
You are using a CUDA device ('NVIDIA GeForce RTX 4090') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision


/home/filippo/miniconda3/envs/peakweather-env/lib/python3.10/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:751: Checkpoint directory /home/filippo/GitHub/peakweather-wind-forecasting/logs exists and is not empty.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name          | Type               | Params | Mode 
-------------------------------------------------------------
0 | loss_fn       | MaskedMAE          | 0      | train
1 | train_metrics | MetricCollection   | 0      | train
2 | val_metrics   | MetricCollection   | 0      | train
3 | test_metrics  | MetricCollection   | 0      | train
4 | model         | TimeThenSpaceModel | 17.5 K | train
-------------------------------------------------------------
17.5 K    Trainable params
0         Non-trainable params
17.5 K    Total params
0.070     Total estimated model params size (MB)
29        Modules in train mode
0         Modules in eval mode


Sanity Checking DataLoader 0:   0%|          | 0/2 [00:00<?, ?it/s]

/home/filippo/miniconda3/envs/peakweather-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.
Only args ['edge_index', 'x', 'edge_weight'] are forwarded to the model (TimeThenSpaceModel).


/home/filippo/miniconda3/envs/peakweather-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Epoch 49: 100%|██████████| 812/812 [00:05<00:00, 135.35it/s, v_num=062a, val_mae=1.920, val_mae_at_15=1.160, val_mae_at_30=1.830, val_mae_at_60=2.170, val_mape=1.36e+5, train_mae=1.930, train_mae_at_15=1.190, train_mae_at_30=1.890, train_mae_at_60=2.210, train_mape=1.36e+5]

`Trainer.fit` stopped: `max_epochs=50` reached.


Epoch 49: 100%|██████████| 812/812 [00:06<00:00, 135.22it/s, v_num=062a, val_mae=1.920, val_mae_at_15=1.160, val_mae_at_30=1.830, val_mae_at_60=2.170, val_mape=1.36e+5, train_mae=1.930, train_mae_at_15=1.190, train_mae_at_30=1.890, train_mae_at_60=2.210, train_mape=1.36e+5]


In [ ]:
#predictor.load_model(checkpoint_callback.best_model_path)
#predictor.freeze()
trainer.test(predictor, datamodule=datamodule);

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/filippo/miniconda3/envs/peakweather-env/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:433: The 'test_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=31` in the `DataLoader` to improve performance.


Testing DataLoader 0: 100%|██████████| 226/226 [00:01<00:00, 203.66it/s]
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
        test_loss           1.9275665283203125
        test_mae            1.9273463487625122
     test_mae_at_15         1.1643688678741455
     test_mae_at_30         1.8550270795822144
     test_mae_at_60          2.211672306060791
        test_mape              152778.953125
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
